ANN (artificial neural networks)
YSA (yapay sinir ağı)

Bu notebook da örnek bir yapay sinir ağı modellemesi yapacağız.
Veri setimiz bir bankanın müşterilerine ait temel kayıtlar. Müşterinin özlük bilgileri ve banka üzerindeki önceki bir kaç kaydı.
Yaşı, Cinsiyeti, Bankadaki total parası, kredi kartı var mı yok mu vb. etiketlerimiz var.
Yapmak istediğimiz şey kişinin bu bilgilerine göre kredi verip vermeyeceğimizi tahmin eden bir model yapmak

Kullanılacak kütüphaneler
numpy - matris işlemleri için
matplotlib - işlem sonuçları ve parametreleri grafiksel olarak göstermek için
pandas - verileri istediğimiz formatlara getirebilmek için

İlk olarak csv formatındaki verilerimizi pandas ile çekelim

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# veriyi çek
dataset = pd.read_csv('Churn_Modelling.csv')
print(dataset.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

Çektiğimiz veriler üzerinde işimize yaramayan veriler var örneğin surname veya customer id gibi bunları kaldıralım

In [2]:
X = pd.DataFrame(dataset.iloc[:, 3:13].values)
y = dataset.iloc[:, 13].values

print(X.head())
print(y)

     0       1       2   3  4          5  6  7  8          9
0  619  France  Female  42  2        0.0  1  1  1  101348.88
1  608   Spain  Female  41  1   83807.86  1  0  1  112542.58
2  502  France  Female  42  8   159660.8  3  1  0  113931.57
3  699  France  Female  39  1        0.0  2  0  0   93826.63
4  850   Spain  Female  43  2  125510.82  1  1  1    79084.1
[1 0 1 ... 1 1 0]


Verimizde text alanlarımızı kategorileyip daha rahat işlem yapabilmek için enum yöntemi gibi integer temsilleri olarak dönüştürelim
2. indexteki kolon 0 ve 1 0 - kadın 1 - erkek
1. indexteki kolon [[0-1],[0-1]] gibi bir matris olarak konum verileri olacak
# [0 0] demek France.
# [0 1] demek Spain
# [1 0] demek Germany

sklearn kütüphanesinden faydalanarak bu etiket dönüşümlerini yapacağız.
Onehotencoder bahsettiğimiz matris dönüşümü görevini üstelenen sınıfımız.

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_2 = LabelEncoder()
X.loc[:, 2] = labelencoder_X_2.fit_transform(X.iloc[:, 2])
labelencoder_X_1 = LabelEncoder()
X.loc[:, 1] = labelencoder_X_1.fit_transform(X.iloc[:, 1])
ct = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')

/var/folders/_k/nhq03yk55h3dw_c3jp61qq5m0000gn/T/ipykernel_43446/1174735350.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[:, 2] = labelencoder_X_2.fit_transform(X.iloc[:, 2])
/var/folders/_k/nhq03yk55h3dw_c3jp61qq5m0000gn/T/ipykernel_43446/1174735350.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[:, 1] = labelencoder_X_1.fit_transform(X.iloc[:, 1])


In [4]:
labelencoder_X_1 = LabelEncoder()
X.loc[:, 1] = labelencoder_X_1.fit_transform(X.iloc[:, 1])
X = ct.fit_transform(X)
X = X[:, 1:]

print(X)

[[0.0 0.0 619 ... 1 1 101348.88]
 [0.0 1.0 608 ... 0 1 112542.58]
 [0.0 0.0 502 ... 1 0 113931.57]
 ...
 [0.0 0.0 709 ... 0 1 42085.58]
 [1.0 0.0 772 ... 1 0 92888.52]
 [0.0 0.0 792 ... 1 0 38190.78]]


Artık girdi verisetimizi tamamen hazırladık. Sıra verisetinin performansını ölçmekte.
Burada sklearn in train_test_split fonksiyonunu kullanarak verisetinin %20 sini eğitip test sonuçlarını alacağız

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

feature scaling
veri setimizde görüldüğü üzere verilerimizin birbirine göre farklı ölçeklerde olduğunu görüyoruz.
özellikle balance ve estimated_salary, bu durum hesap işlemlerimizi daha uzun sürede yapmamıza neden oluyor.
feature scaling bize verilerimizi belirli bir ölçekte normalize etmemizi sağlıyor.

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Artık verinin preprocessing(ön işleme) aşamalarını tamamladık. Şimdi sıra yapay sinir ağımızı oluşturmada.
Keras(tensorflow) kütüphanesini kullanarak bir yapay sinir ağı oluşturacağız

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Sequential class'i ile bir YSA oluşturuyoruz.
Sequential classı yapay sinir ağlarını(sıralı katman) oluşturmak için kullanılır


In [8]:
classifier = Sequential()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-30 16:22:08.106825: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-30 16:22:08.107179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Giriş katmanı ve ilk gizli katmanımızı oluşturuyoruz.
Dense tensorflow da en sık kullanılan classdır. Dense ANN ye tam bağlı bir katman eklememizi sağlar.
add metodu ile katmanı ekliyoruz.
output_dim = 6, gizli katmandaki nöron sayısını temsil eder. burada 6 dememizin çok bir anlamı yok
activation = 'relu' ise gizli katmanın aktivasyon fonksiyonunu temsil eder. relu fonksiyonu gizli katmanın aktivasyon fonksiyonu olarak kullanılır.
input_dim = 11 ise girdideki bağımsız değişkenlerimizi temsil eder. 

In [9]:
classifier.add(Dense(6, 'relu', input_dim = 11))   

ikinci gizli katmanı oluştur

In [10]:
classifier.add(Dense(6, 'relu'))

çıkış katmanı oluştur
aktivasyon fonksiyonu sigmoid

In [11]:
classifier.add(Dense(1, 'sigmoid'))

 yapay sinir ağı derle
adam stochastic gradient descent algoritmasını kullanır.
optimizer eğitim boyunca ağırlıkları günceller ve kaybı azaltır.
binary prediction yapmak istediğimiz için loss u binary_crossentropy olarak belirledik.
metrics = ['accuracy'] ile doğruluk metrikleri kullanacağımızı belirttik.

In [12]:
classifier.compile('adam', 'binary_crossentropy', ['accuracy'])

yapay sinir ağımızı eğitiyoruz

In [13]:
classifier.fit(X_train, y_train, 10, 100)

2022-12-30 16:22:08.325459: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2022-12-30 16:22:08.505504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


800/800 [==============================] - 4s 5ms/step - loss: 0.5329 - accuracy: 0.7630
Epoch 2/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4365 - accuracy: 0.7964
Epoch 3/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4176 - accuracy: 0.8135
Epoch 4/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4079 - accuracy: 0.8200
Epoch 5/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4005 - accuracy: 0.8241
Epoch 6/100
800/800 [==============================] - 4s 5ms/step - loss: 0.3931 - accuracy: 0.8294
Epoch 7/100
800/800 [==============================] - 4s 5ms/step - loss: 0.3854 - accuracy: 0.8360
Epoch 8/100
800/800 [==============================] - 4s 5ms/step - loss: 0.3774 - accuracy: 0.8405
Epoch 9/100
800/800 [==============================] - 4s 5ms/step - loss: 0.3712 - accuracy: 0.8449
Epoch 10/100
800/800 [==============================] - 4s 5ms/step - loss: 0.3647 - accuracy: 0.8489
E

eğitilmiş veriye göre tahmin yapıyoruz

In [14]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

63/63 [==============================] - 0s 2ms/step


2022-12-30 16:28:33.406517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


confusion matrix ve accuracy score hesapla
confusion matrix kullanmamızın sebebi tahminlerimizin doğruluğunu ölçmek için daha iyi bir gösterim sağlamasıdır.
0 ve 1 arasında bir doğruluk payı döner

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [16]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
acc_sc = accuracy_score(y_test,y_pred)
print("Accuracy score")
print(acc_sc)

[[1507   88]
 [ 199  206]]
Accuracy score
0.8565
